In [ ]:

if __name__ == '__main__':
    %run MoleculeGenerator2.ipynb
#    %run Discrim.ipynb
 #   %run ChemEnv.ipynb
    %run MolUtils.ipynb
    #%run Tests.ipynb

    
from torch.utils.tensorboard import SummaryWriter
from time import time
from torch.optim import Adam
import os
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class GraphDataLoader():
    def __init__(self,path,batch_size, length):
        self.path = path
        self.length = length
        
        self.batch_size = batch_size
        self.indicies = list(range(length))
        random.shuffle(self.indicies)
        
        self.curr_idx = 0
        
    def __get_exs(self,indices):
        graphs, graph_dict = dgl.load_graphs(self.path, indices)
        last_action = torch.stack([graph_dict['last_action'][idx] for idx in indices],dim = 0)
        last_atom_feat = torch.stack([graph_dict['last_atom_feats'][idx] for idx in indices], dim = 0)
        action = torch.stack([graph_dict['actions'][idx] for idx in indices], dim = 0)
        graphs = dgl.batch(graphs)
        return graphs.to(device), torch.unsqueeze(last_action,dim=1).to(device), last_atom_feat.to(device), action.to(device)
    
    def __next__(self):
        if self.curr_idx + self.batch_size > self.length:
            self.curr_idx = 0 
            random.shuffle(self.indicies)       
        batch_indices = self.indicies[self.curr_idx: self.curr_idx+self.batch_size]
        self.curr_idx += self.batch_size
    
        return self.__get_exs(batch_indices)
        
    
        

In [ ]:



# import matplotlib.pyplot as plt
# import numpy as np

In [ ]:
# graphs, graph_dict = dgl.load_graphs('./graph_decomp/full_chunka', [i for i in range(5000)])

In [ ]:
# l = [int(i) for i in (graph_dict['actions'].tolist())]

In [ ]:
# print(4)
# plt.hist(l, bins=30)

In [ ]:
class SupervisedTrainingWrapper():
    def __init__(self, input_dim, batch_size, num_atom_types, writer, data_set_size, path = './graph_decomp/full_chunka'):
        #svt = SupervisedTrainingWrapper(54,5,15,None)
        self.path = path        
        self.policy = BaseLine(input_dim,300,17+1).cuda()
        self.optim = Adam(self.policy.parameters(), lr=3e-4)
        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.chunk_idx = 0
        
        self.writer = writer
        self.n_iter = 0
        self.cv_iter = 0
        
        self.batch_size = batch_size
        self.data_set_size = data_set_size
        self.dataIter = GraphDataLoader(path,batch_size,data_set_size)
        print(data_set_size)

    def CalcAccuracy(self):
        a,b,c,d = next(self.dataIter)
        return self.calc_accuracy(a,b,c,d)
    
    
    def calc_accuracy(self,graphs, last_action_nodes, last_atom_feats, actions):
        y_logits = self.policy(graphs, last_action_nodes, last_atom_feats)
        values, pred_labels = y_logits.max(dim=1)
        acc = sum(pred_labels == actions)/actions.size()[0]
        return acc
    
    def calc_accuracy_no_call(self,logits,actions):
        values, pred_labels = logits.max(dim=1)
        acc = sum(pred_labels == actions)/actions.size()[0]
        return acc
        
        
    def Train(self,num_epochs):
        steps_per_epoch = self.data_set_size//self.batch_size
        
        
        for i in range(num_epochs):
            acc = self.CalcAccuracy()
            t0 = time()
            for step in range(steps_per_epoch):
                self._train()
            self._train()
            t1 = time()
            
            print(f'Time for epoch {i} is {t1-t0}, random accuracy is {acc}')
            
            
    def _train(self,calc_accuracy = True, update = True):
        graphs, last_action_nodes, last_atom_feats, actions = next(self.dataIter)
        
        pred = self.policy.forward(graphs, last_action_nodes,last_atom_feats, softmax=False)
        loss = self.loss_fn(pred,actions.long())
        
        if calc_accuracy:
            acc = self.calc_accuracy_no_call(pred,actions)
            self.writer.add_scalar("Accuracy", acc, self.n_iter)
        
        if update:
            self.optim.zero_grad()
            loss.backward()
            self.optim.step()
            
        self.writer.add_scalar("pre_train_loss", loss.detach(), self.n_iter)
        self.n_iter += 1



In [ ]:
# # svt = SupervisedTrainingWrapper(54, 2, 17, None, 10000)
# # svt.Train(2)
# a,b,c,d = (next(svt.dataIter))
# print(a)
# svt.calc_accuracy(a,b,c,d)

In [ ]:
# gd = GraphDataLoader('./graph_decomp/full_chunka',10,500000)

In [ ]:
# %run MoleculeGenerator2.ipynb
# svt = SupervisedTrainingWrapper(54, 100, 17, None, 10000)

In [ ]:
# svt.Train(9)

In [ ]:
class Supervised_Trainer(SupervisedTrainingWrapper):
    def __init__(self,policy_model, **kwargs):
        self.policy = policy_model
        super().__init__(**kwargs) 
        
        
    def TrainModel(self,total_epochs):
        
        self.Train(total_epochs)
        return self.policy